In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
!cp /content/drive/MyDrive/2-folder/kaggle/df_utils.py /content/
!cp /content/drive/MyDrive/2-folder/kaggle/nn_utils.py /content/
import df_utils
import nn_utils

In [4]:
df_train_loaded = pd.read_csv('/content/drive/MyDrive/2-folder/kaggle/housing-prices-competition/train.csv')
df_test_loaded = pd.read_csv('/content/drive/MyDrive/2-folder/kaggle/housing-prices-competition/test.csv')
print(f"df_train_loaded = {df_train_loaded.shape}, df_test_loaded = {df_test_loaded.shape}")

# num_col = len(df_train.columns)
# print(f"num_col = {num_col}")
# print(df_train["SalePrice"][:5])

df_train = df_train_loaded.drop(['Id'], axis=1)
df_test = df_test_loaded.drop(['Id'], axis=1)

# Drop all columns with with more than 20% of missing values
df_utils.drop_col_miss_val(df_train, df_test, 20)

# Fix categorical values and missing values
df_train, df_test = df_utils.prepare_df(df_train, df_test, 5, ['SalePrice'])
# print(df_train)

# Calculate the correlation of features with the target
correlation = df_train.corr()
sorted_corr = correlation['SalePrice'].sort_values(ascending=False)
# print(sorted_corr)
columns = []
for i, v in sorted_corr.items():
  if v > 0.2 and i != 'SalePrice':
    # print('index: ', i, 'value: ', v)
    columns.append(i)
# print(columns)

# Remove target label from training set
y_train = df_train['SalePrice'].values
# print(y[0:5])
df_train = df_train.drop(['SalePrice'], axis=1)

# Extract columns with high correlaton
df_train = df_train[columns]
df_test = df_test[columns]
# print(len(df_train.columns))
# print(len(df_test.columns))
# print(df_train.head(5))

# normalize dataframe
mean = df_train.mean()
std = df_train.std()
df_train=(df_train-mean)/std
df_test=(df_test-mean)/std

X = df_train.values
y = y_train

# create train, validation and test splits
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)
print(f"X_train = {X_train.shape}, y_train = {y_train.shape}")
print(f"X_val = {X_val.shape}, y_val = {y_val.shape}")
print(f"X_test = {X_test.shape}, y_test = {y_test.shape}")

df_train_loaded = (1460, 81), df_test_loaded = (1459, 80)
X_train = (876, 36), y_train = (876,)
X_val = (292, 36), y_val = (292,)
X_test = (292, 36), y_test = (292,)


In [6]:
# Convert data to PyTorch tensors
X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.float32)

X_val_t = torch.tensor(X_val, dtype=torch.float32)
y_val_t = torch.tensor(y_val, dtype=torch.float32)

# Define the model
class RegressionModel(nn.Module):
    def __init__(self, input_dim):
        super(RegressionModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32*4)
        self.fc3 = nn.Linear(32*4, 32*4)
        self.fc4 = nn.Linear(32*4, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        return x

input_dim = X_train_t.shape[1]
model_0 = RegressionModel(input_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(model_0.parameters(), lr=0.001)

nn_utils.training_loop(epochs=800, batch_size=30, model_0, criterion, optimizer, X_train_t, y_train_t, X_val_t, y_val_t)

Epoch [200/800], Train Loss: 1215140736.0000, Test Loss: 989913984.0000
Epoch [400/800], Train Loss: 301810560.0000, Test Loss: 972856128.0000
Epoch [600/800], Train Loss: 37966936.0000, Test Loss: 1039949120.0000
Epoch [800/800], Train Loss: 76560104.0000, Test Loss: 1058605696.0000
